In [1]:
# TODO
# add latest jersey numbers DONE
# list jersey numbers per season DONE
# add career stats DONE
# add year by year stats that are availble
# season leaders DONE

In [2]:
import re
import pandas as pd
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:
        row=i.split('\t')
        if(len(row[0])>7 and len(row)==1 and ('COACH' not in i and 'OVERALL' not in i and 'ECAC:' not in i and 'CAPTAIN' not in i and 'HOCKEY' not in i and 'NEIHL:' not in i and 'forfeit' not in i)):
            tourneys.append(row[0])
            
            
tourneyDict={}
for i in tourneys:
    if(i=='Key to Tournaments'):
        continue
    tourney=i.split(' ')
    tourneyDict[tourney[0]]=' '.join(tourney[1:])
tourneyDict['nc']='Non-Colleigate'
tourneyDict['Oly nc']='1932 NEAAU Olympic tryouts-Non-Colleigate'
tourneyDict['ex']='Exhibition'
tourneyDict['HF ex']='Hall of Fame Game-Exhibition'
tourneyDict['IB ex']='Ice Breaker-Exhibition'

In [3]:
import re
from datetime import datetime
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
gameList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:      
        if(len(i)==7):
            season=i
        coachSearch=re.search("COACH: (.*)",i)
        if coachSearch != None:
            coach=coachSearch.group(1)
        captSearch=re.search("CAPTAIN.?: (.*)",i)
        if captSearch != None:
            capt=captSearch.group(1) 
            #print(capt)
        confSearch=re.search("(NEIHL|ECAC|HOCKEY EAST):",i)
        if confSearch != None:
            conf=confSearch.group(1)
            if(conf=='HOCKEY EAST'):
                conf='Hockey East'
        if(re.search('\*',i)!=None):
            gameType='Conference'
        else:
            gameType='Non-Conference'
        i=i.replace("* ",'')
        note=''
        if('†' in i):
            note='Loss by forfeit (ineligible player)'
            i=i.replace('†','')
        if('+' in i):
            note='Win by forfeit (opponent left ice – score was 5-1, BU)'
            i=i.replace('+','')
        if('‡' in i):
            note='Win by forfeit (ineligible player)'
            i=i.replace('‡','')
        game=re.search(r"(\d*\/\d*) (\w*) (?:\((.?ot)\))? ?(.*)\t(\S*|\S* \S*|\S* \S* \S*) ?(\(.*\))? (\d*-\d*)",i)
        if game==None:
            continue
        #print(game.groups())
        gameDict={'date':game.group(1),
                 'result':game.group(2),
                 'ot':game.group(3),
                 'arena':game.group(4),
                 'opponent':game.group(5),
                 'gameType':gameType,
                 'tourney': game.group(6),
                 'scoreline':game.group(7),
                 'location':'',
                 'coach':coach,
                 'captain':capt,
                 'conference':conf,
                 'season':season,
                 'note':note}
        if(gameDict['gameType']==None):
            gameDict['gameType']='Non-Conference'
        if(gameDict['tourney']=='(ex)' or gameDict['result']=='E'):
            gameDict['gameType']='Exhibition'
            gameDict['tourney'] = None
        if(gameDict['arena']=='Agganis Arena' or gameDict['arena']=='Brown Arena' or gameDict['arena']=='Boston Arena'):
            gameDict['location']='Home'
        elif(gameDict['tourney']==None or gameDict['tourney']=='(nc)' or gameDict['tourney'] == '(B1G/HE)' or ((gameDict['tourney'] == '(HE)' or gameDict['tourney'] == '(ECAC)') and (gameDict['arena'] != 'TD Garden' and gameDict['arena'] != 'Boston Garden' and gameDict['arena'] != 'Providence CC'))):
            gameDict['location']='Away'
        if(gameDict['location']=='' or gameDict['arena']=='Boston Garden' or gameDict['arena']=='VW Arena'):
            gameDict['location']='Neutral'
        if((gameDict['arena']=='Gutterson' and gameDict['opponent']=='Vermont') or (gameDict['arena']=='Houston' and gameDict['opponent']=='Rensselaer') or (gameDict['arena']=='Broadmoor' and gameDict['opponent']=='Colorado College') or (gameDict['arena']=='DEC Center' and gameDict['opponent']=='Minnesota Duluth')or (gameDict['arena']=='Magness Arena' and gameDict['opponent']=='Denver')or (gameDict['arena']=='Mariucci Arena' and gameDict['opponent']=='Minnesota')or (gameDict['arena']=='Munn Ice Arena' and gameDict['opponent']=='Michigan State')or (gameDict['arena']=='Walker Arena' and gameDict['opponent']=='Clarkson')or (gameDict['arena']=='Thompson Arena' and gameDict['opponent']=='Dartmouth')or (gameDict['arena']=='St. Louis Arena' and gameDict['opponent']=='St. Louis') or (gameDict['arena']=='Sullivan Arena' and gameDict['opponent']=='Alaska Anchorage')):
            gameDict['location']='Away'

        if(gameDict['tourney']!=None):
            #print(gameDict)
            gameDict['tourney']=tourneyDict[gameDict['tourney'].replace('(','').replace(')','')]
        gameDict['month'],gameDict['day']=gameDict['date'].split('/')
        if(int(gameDict['month']) >=9):
            gameDict['date']+="/" + gameDict['season'][:4]
            gameDict['year']=int(gameDict['season'][:4])
        elif(int(gameDict['month']) <= 4):
            gameDict['date']+= "/" + str(int(gameDict['season'][:4])+1)
            gameDict['year']=int(gameDict['season'][:4])+1
        gameDict['BUScore'],gameDict['OppoScore']=int(gameDict['scoreline'].split('-')[0]),int(gameDict['scoreline'].split('-')[1])
        gameDict['GD']=abs(gameDict['BUScore']-gameDict['OppoScore'])
        if(gameDict['season']=='1973-74' and gameDict['date']=='12/12/1973'):
            coach='Jack Parker'
        gameDict['dow']=datetime(gameDict['date']).weekday()
        gameList.append(gameDict)
        teamSet.add(gameDict['opponent'])
f.close()
df=pd.DataFrame(gameList)

TypeError: an integer is required (got type str)

In [ ]:
df

In [ ]:
teams="Air Force|Alaska Anchorage|American International|Arizona State|Army|Bates|Bentley|Boston College|Bowdoin|Bowling Green|Brown|Clarkson|Colby|Colgate|Colorado College|Connecticut|Cornell|Dalhousie|Dartmouth|Denver|Ferris State|Hamilton College|Harvard|Holy Cross|Lake Superior State|Loyola of Montreal|Maine|Massachusetts|Massachusetts Devens|UMass Lowell|Memorial University|Merrimack|Miami|Michigan|Michigan State|Michigan Tech|Middlebury|Minnesota|Minnesota Duluth|Minnesota State|MIT|Nebraska-Omaha|New Brunswick|New Hampshire|Niagara|North Dakota|Northeastern|Northern Arizona|Northern Michigan|Norwich|Notre Dame|Ohio State|Ohio University|Pennsylvania|Princeton|Providence|Queen’s|Quinnipiac|Rensselaer|Robert Morris|Royal Military College|Sacred Heart|St. Cloud State|St. Francis Xavier|St. Lawrence|St. Louis|Toronto|Tufts|Union|US International|Vermont|Western Michigan|Williams|Wisconsin|Yale|Intercollegiate Totals|Non-Collegiate|Boston Athletic Assoc.|Boston Hockey Club|Brae Burn|Briarcliff Lodge|Eastern Nationals|Jr. Olympics|L’Assoc. St. Dominique|Maple Athletic Assoc.|Needham Hockey Club|New Haven Club|Nichols Club|St. Nicholas|Tech Beavers|Union Boat Club|University Club|U.S. National Team"

In [4]:
print('["',end='')
for i in range(2002,2021):
    print(i,str(i+1)[2:],sep='-',end='","')
    
print('\b\b]')

["2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21","]


In [5]:
for s in ['2018-19','2019-20','2020-21']:
#for s in ["2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13"]:

    print(s)

    resSum={'W':0,'L':0,'T':0}
    for i in ['Home','Away','Neutral']:
        print(i,end=":")
        for d in ['W','L','T']:
            res=df.loc[(df['location']==i) & (df['season']==s) & (df['result']==d) & (df['tourney']!='Exhibition')]['date'].count()
            print(res,end='-')
            resSum[d]+=res
        print("\b")
    print("Total: {}-{}-{}".format(resSum['W'],resSum['L'],resSum['T']))

2018-19
Home:

NameError: name 'df' is not defined

In [6]:
#df.loc[(df['season']=='2012-13') & (df['location']!='Home')]

In [7]:
x=df.loc[(df['location']!="Home")].groupby(['arena','opponent','location']).size()

NameError: name 'df' is not defined

In [8]:
df['year']

NameError: name 'df' is not defined

In [9]:
df.loc[(df['coach']=="Albie O'Connell")].sort_values('GD',ascending=False)

NameError: name 'df' is not defined

In [10]:
(df.loc[(df['season']=="2004-05") & (df['location']!="Home")])

NameError: name 'df' is not defined

In [11]:
x=df.groupby(['arena','opponent','location']).size()

NameError: name 'df' is not defined

In [12]:
x=df.loc[(df['location']!="Home")].groupby(['arena','opponent','location']).size()
rows=(x.to_string().split('\n'))
for i in rows:
    print(repr(i))

NameError: name 'df' is not defined

In [13]:
df.loc[(df['coach']=="David Quinn")  & (df['season']=='2013-14')  & (df['result']=='W')  &  (df['tourney']!='Exhibition')].count()

NameError: name 'df' is not defined

In [14]:
locs=set()
for i in gameList:
    locs.add(i['arena'])

In [15]:
fileName=('JerseyNumbers.txt')
tourneys=[]
teamSet=set()
playerList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    numDict={'number':0,'player':'','season':''}
    for i in rows:
        
        numSearch=re.search("#(.*)",i)
        if numSearch != None:
            number=numSearch.group(1)
        if("Retired - " in i):
                continue
        playerSearch=re.search("(\d*-\d*) (.*)",i)
        if playerSearch != None:
            season=playerSearch.group(1)
            numDict={'number':int(number),
                 'season':convertSeasons(season),
                 'player':playerSearch.group(2)}
            playerList.append(numDict)
f.close()

NameError: name 'convertSeasons' is not defined

In [16]:
jerseydf=pd.DataFrame(playerList)

In [17]:
jerseydf

""


In [18]:
jerseydf.loc[jerseydf.season.str.contains('2022-23')]

AttributeError: 'DataFrame' object has no attribute 'season'

In [19]:
def convertToInt(val):
        if(val.isdigit()):
            val=int(val)
        else:
            val=None
        return val

In [20]:
def convertToFloat(val):
        try:
            val=float(val)
        except:
            val=float('nan')
        return val

In [21]:
def convertSeasons(season):
        gap=season.split(',')
        years=season[2:].split('-')
        seasonStr=''
        if(len(gap)>1):
            for i in gap:
                seasonStr+=convertSeasons(i)
        else:
            yearDiff=abs(int(years[0])-int(years[1]))
            if(yearDiff>6):
                yearDiff=100-yearDiff
            firstHalf=season[:4]
            seasonStr=''
            for i in range(yearDiff):
                secondHalf=int(firstHalf)+1
                seasonStr+=str(firstHalf)+'-'+str(secondHalf)[2:]+','
                firstHalf=secondHalf
        return seasonStr[:-1]

In [22]:
fileName=('SkaterStats.txt')
tourneys=[]
teamSet=set()
skateList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        skaterSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)\/(\S*)',i)
        if skaterSearch!=None:
            skaterDict={'last':skaterSearch.group(1),
                       'first':skaterSearch.group(2),
                        'name':skaterSearch.group(2)+' '+skaterSearch.group(1),
                       'seasons':convertSeasons(skaterSearch.group(3)),
                       'gp':convertToInt(skaterSearch.group(4)),
                       'goals':convertToInt(skaterSearch.group(5)),
                       'assists':convertToInt(skaterSearch.group(6)),
                       'pts':convertToInt(skaterSearch.group(7)),
                       'pen':convertToInt(skaterSearch.group(8)),
                       'pim':convertToInt(skaterSearch.group(9))}
            skateList.append(skaterDict)
dfSkate=pd.DataFrame(skateList)

In [23]:
fileName=('GoalieStats.txt')
tourneys=[]
teamSet=set()
goalieList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        goalieSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)',i)
        if goalieSearch!=None:
            mins=goalieSearch.group(5).split(':')
            if(len(mins)>1):
                time = "{}:{}".format(*divmod(int(mins[0]), 60)) + ":" + mins[1]
                time = pd.to_timedelta(time)
            else:
                time=float('nan')
            goalieDict={'last':goalieSearch.group(1),
                       'first':goalieSearch.group(2),
                       'name':goalieSearch.group(2)+' '+goalieSearch.group(1),
                       'seasons':convertSeasons(goalieSearch.group(3)),
                       'gp':convertToInt(goalieSearch.group(4)),
                       'mins':round(pd.Timedelta(time).total_seconds()/60,2),
                       'ga':convertToInt(goalieSearch.group(6)),
                       'gaa':convertToFloat(goalieSearch.group(7)),
                       'saves':convertToInt(goalieSearch.group(8)),
                       'sv%':convertToFloat(goalieSearch.group(9)),
                       'W':convertToInt(goalieSearch.group(10)),
                       'L':convertToInt(goalieSearch.group(11)),
                       'T':convertToInt(goalieSearch.group(12))}
            goalieList.append(goalieDict)
dfGoalie=pd.DataFrame(goalieList)            

In [24]:
dfSkate=pd.DataFrame(skateList)
dfSkate.loc[(dfSkate['seasons'].str.contains('pres'))]

,last,first,name,seasons,gp,goals,assists,pts,pen,pim


In [25]:
dfSkate

,last,first,name,seasons,gp,goals,assists,pts,pen,pim
0,Abbott,Darrell,Darrell Abbott,"1966-67,1967-68,1968-69",92.0,14,50,67,NaN,34.0
1,Abbott,Larry,Larry Abbott,"1970-71,1971-72,1972-73",62.0,4,10,14,5.0,17.0
2,Ahola,Peter,Peter Ahola,"1989-90,1990-91",82.0,15,44,59,68.0,153.0
3,Aiken,John,John Aiken,1954-55,NaN,0,0,0,NaN,4.0
4,Alger,Rich,Rich Alger,"1995-96,1996-97",35.0,1,5,6,15.0,41.0
...,...,...,...,...,...,...,...,...,...,...
782,Amonte,Tristan,Tristan Amonte,2021-22,17.0,1,2,3,5.0,10.0
783,Boucher,Tyler,Tyler Boucher,2021-22,17.0,2,1,3,13.0,34.0
784,Gallagher,Ty,Ty Gallagher,2021-22,34.0,5,11,16,12.0,35.0
785,Page,Jack,Jack Page,2021-22,1.0,0,0,0,0.0,0.0


In [26]:
x=jerseydf.loc[jerseydf.season.str.contains('2021-22')]['player']

AttributeError: 'DataFrame' object has no attribute 'season'

In [27]:
jerseydf.loc[jerseydf.season.str.contains('2019-20')]

AttributeError: 'DataFrame' object has no attribute 'season'

In [28]:
for i in x:
    if(dfSkate.loc[dfSkate['name']==i]['name'].count()==0 and dfGoalie.loc[dfGoalie['name']==i]['name'].count()==0):
        print(i)


NameError: name 'x' is not defined

In [29]:
dfSkate

,last,first,name,seasons,gp,goals,assists,pts,pen,pim
0,Abbott,Darrell,Darrell Abbott,"1966-67,1967-68,1968-69",92.0,14,50,67,NaN,34.0
1,Abbott,Larry,Larry Abbott,"1970-71,1971-72,1972-73",62.0,4,10,14,5.0,17.0
2,Ahola,Peter,Peter Ahola,"1989-90,1990-91",82.0,15,44,59,68.0,153.0
3,Aiken,John,John Aiken,1954-55,NaN,0,0,0,NaN,4.0
4,Alger,Rich,Rich Alger,"1995-96,1996-97",35.0,1,5,6,15.0,41.0
...,...,...,...,...,...,...,...,...,...,...
782,Amonte,Tristan,Tristan Amonte,2021-22,17.0,1,2,3,5.0,10.0
783,Boucher,Tyler,Tyler Boucher,2021-22,17.0,2,1,3,13.0,34.0
784,Gallagher,Ty,Ty Gallagher,2021-22,34.0,5,11,16,12.0,35.0
785,Page,Jack,Jack Page,2021-22,1.0,0,0,0,0.0,0.0


In [30]:
fileName=('GoalieStats.txt')
tourneys=[]
teamSet=set()
goalieList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        goalieSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)',i)
        if goalieSearch!=None:
            mins=goalieSearch.group(5).split(':')
            if(len(mins)>1):
                time = "{}:{}".format(*divmod(int(mins[0]), 60)) + ":" + mins[1]
                time = pd.to_timedelta(time)
            else:
                time=float('nan')
            goalieDict={'last':goalieSearch.group(1),
                       'first':goalieSearch.group(2),
                       'name':goalieSearch.group(2)+' '+goalieSearch.group(1),
                       'seasons':convertSeasons(goalieSearch.group(3)),
                       'gp':convertToInt(goalieSearch.group(4)),
                       'mins':round(pd.Timedelta(time).total_seconds()/60,2),
                       'ga':convertToInt(goalieSearch.group(6)),
                       'gaa':convertToFloat(goalieSearch.group(7)),
                       'saves':convertToInt(goalieSearch.group(8)),
                       'sv%':convertToFloat(goalieSearch.group(9)),
                       'W':convertToInt(goalieSearch.group(10)),
                       'L':convertToInt(goalieSearch.group(11)),
                       'T':convertToInt(goalieSearch.group(12))}
            goalieList.append(goalieDict)
dfGoalie=pd.DataFrame(goalieList)            

In [31]:
dfGoalie=pd.DataFrame(goalieList)

In [356]:
seasons=["2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]
for season in seasons:
    print(season[2:4]+season[-2:])

0203
0304
0405
0506
0607
0708
0809
0910
1011
1112
1213
1314
1415
1516
1617
1718
1819
1920
2021


In [32]:
import urllib.request, urllib.error, urllib.parse
from bs4 import BeautifulSoup
import re

skaterSeasonList=[]
goalieSeasonList=[]
#seasons=["2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14","2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]
seasons=["2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]

for season in seasons:
    sSeason=season[2:4]+season[-2:]
    url = "http://www.collegehockeystats.net/{}/teamstats/bu_m".format(sSeason)
    #print(url)
    f=urllib.request.urlopen(url)
    html = f.read()
    f.close()
    soup = BeautifulSoup(html, 'html.parser')
    tab = soup.find_all("table")
    skatersTab=tab[2]
    tabRows=skatersTab.find_all('tr')
    statsDict={"number" : 0,
    "name" : 1,
    "pos" : 2,
    "yr" : 3,
    "overall_gp" : 4,
    "overall_goals" : 5,
    "overall_assists" : 6,
    "overall_pts" : 7,
    "overall_pen" : 8,
    "overall_ppg" : 9,
    "overall_shg" : 10,
    "overall_gwg" : 11,
    "overall_plusminus" : 12,
    "overall_sog" : 13,
    "conf_gp" : 14,
    "conf_goals" : 15,
    "conf_assists" : 16,
    "conf_pts" : 17,
    "conf_pen" : 18,
    "conf_ppg" : 19,
    "conf_shg" : 20,
    "conf_gwg" : 21,
    "conf_plusminus" : 22,
    "conf_sog" : 23,
    "career_gp" : 24,
    "career_goals" : 25,
    "career_assists" : 26,
    "career_pts" : 27}

    gStatsDict={"number" : 0,
    "name" : 1,
    "yr" : 2,
    "gp" : 3,
    "mins" : 4,
    "ga" : 5,
    "saves" : 6,
    "shots" : 7,
    "saveperc" : 8,
    "gaa" : 9,
    "record" : 10,
    "winpec" : 11,
    "gs" : 12,
    "so" : 13}

    skateRows=skatersTab.find_all('tr', class_=lambda x: x != 'chssmallbold')
    for i in range(1,len(skateRows)-1):
        playerDict={}
        for d in statsDict.keys():
                playerDict[d] = skateRows[i].find_all('td')[statsDict[d]].get_text().lstrip(' ').rstrip(' ').replace('\xa0','N/A')
        playerDict['name']=re.sub(' \(.*','',playerDict['name'])
        playerDict['season']=season

        skaterSeasonList.append(playerDict)


    goalieTab=tab[3]
    goalieRows=goalieTab.find_all('tr', class_=lambda x: x != 'chssmallbold')

    for i in range(1,len(goalieRows)):
        playerDict={}
        if("Open Net" in goalieRows[i].get_text() or "##" in goalieRows[i].get_text()):
            break
        for d in gStatsDict.keys():
            playerDict[d] = goalieRows[i].find_all('td')[gStatsDict[d]].get_text().lstrip(' ').rstrip(' ').replace('\xa0','N/A')
        playerDict['name']=re.sub(' \(.*','',playerDict['name'])
        playerDict['season']=season
        goalieSeasonList.append(playerDict)



In [33]:
dfSkateSeason=pd.DataFrame(skaterSeasonList)
dfGoalSeason=pd.DataFrame(goalieSeasonList)

In [141]:
dfSkateSeason[['number','name','pos','yr','overall_gp','overall_goals','overall_assists','overall_pts','season']].to_string(index=False)

"number                      name  pos  yr overall_gp overall_goals overall_assists overall_pts   season\n     9               Jack Eichel    F  FR         40            26              45          71  2014-15\n    17            Evan Rodrigues    F  SR         41            21              40          61  2014-15\n    10             Danny O'Regan    F  JR         41            23              27          50  2014-15\n     2              Ahti Oksanen  F/D  JR         40            25              13          38  2014-15\n     5             Matt Grzelcyk    D  JR         41            10              28          38  2014-15\n     7             Cason Hohmann    F  SR         41            11              20          31  2014-15\n    21                 Matt Lane    F  JR         41             8              10          18  2014-15\n    25         Brandon Fortunato    D  FR         41             1              17          18  2014-15\n     4            Brandon Hickey    D  FR         41  

In [35]:
x=jerseydf.loc[jerseydf.season.str.contains('2019-20|2020-21|2021-22|2022-23')]['player']
for i in x:
    if(dfSkate.loc[dfSkate['name']==i]['name'].count()==0 and dfGoalie.loc[dfGoalie['name']==i]['name'].count()==0):
        #print(dfSkateSeason.loc[(dfSkateSeason['name']==i)][['name','season','overall_gp','overall_goals','overall_assists','overall_pts','overall_pen']].to_string())
        #if(dfGoalie.loc[dfGoalie['name']==i]['name'].count()>0):
            print(i,dfGoalSeason.loc[(dfGoalSeason['name']==i)][['name','season','gp','mins','ga','gaa','shots','saveperc','record']].to_string())
    

AttributeError: 'DataFrame' object has no attribute 'season'

In [ ]:
### Current Team#####

In [206]:
url = "https://goterriers.com/sports/mens-ice-hockey/stats?path=mhockey"
#print(url)
f=urllib.request.urlopen(url)
html = f.read()
f.close()
soup = BeautifulSoup(html, 'html.parser')

In [207]:
curSkate=soup.find('section',{'id':'individual-overall-skaters'})

In [208]:
rows=curSkate.find_all('tr')
currSkaters=[]
for i in rows:
    col=i.find_all('td')
    name=i.find('span')
    
    if(name!=None):
        counter=0
        lastName,firstName=name.get_text().split(', ')
        skateDict={'number':int(col[0].get_text()),
                   'last':lastName,
                   'first':firstName,
                   'name':firstName+' '+lastName,
                    'gp':int(col[1].get_text()),
                    'goals':int(col[2].get_text()),
                    'assists':int(col[3].get_text()),
                    'pts':int(col[4].get_text()),
                    'pen':col[16].get_text().replace('-','/')}
        currSkaters.append(skateDict)

In [209]:
dfCurr=pd.DataFrame(currSkaters)

In [210]:
dfCurr[['number','name','gp','goals','assists','pts','pen']].to_string(index=False).split('\n')

[' number                 name  gp  goals  assists  pts    pen',
 '     23     Domenick Fensore  35      5       26   31    4/8',
 '     32         Wilmer Skoog  35     15       15   30  10/20',
 '     16  Robert Mastrosimone  34     11       14   25  10/28',
 "     18          Jay O'Brien  24     10       12   22  15/38",
 '     37           Matt Brown  24      7       10   17    2/4',
 '     28       Ethan Phillips  28      6       11   17   6/12',
 '     13       Dylan Peterson  29     10        6   16   9/18',
 '      4         Ty Gallagher  34      5       11   16  12/35',
 '      2        Case McCarthy  26      5        9   14   6/20',
 '     21          Max Kaufman  34      6        5   11   6/12',
 '      9      Logan Cockerill  30      5        6   11    4/8',
 '     11            Luke Tuch  26      6        4   10   8/16',
 '     10         Nick Zabaneh  29      1        9   10   7/14',
 '     22       Brian Carrabes  24      4        4    8    2/4',
 '      7          Alex V

In [211]:
for i in dfCurr['name']:
    if(dfSkate.loc[(dfSkate['name']==i)]['name'].count()>0):
        print(dfSkate.loc[(dfSkate['name']==i)].to_string())
        print(dfCurr.loc[(dfCurr['name']==i)].to_string())

        last     first              name                  seasons    gp  goals  assists  pts   pen   pim
769  Fensore  Domenick  Domenick Fensore  2019-20,2020-21,2021-22  85.0     10       45   55  20.0  48.0
   number     last     first              name  gp  goals  assists  pts  pen
0      23  Fensore  Domenick  Domenick Fensore  35      5       26   31  4/8
      last   first          name                  seasons    gp  goals  assists  pts   pen   pim
776  Skoog  Wilmer  Wilmer Skoog  2019-20,2020-21,2021-22  66.0     24       21   45  21.0  42.0
   number   last   first          name  gp  goals  assists  pts    pen
1      32  Skoog  Wilmer  Wilmer Skoog  35     15       15   30  10/20
             last   first                 name                  seasons    gp  goals  assists  pts   pen   pim
771  Mastrosimone  Robert  Robert Mastrosimone  2019-20,2020-21,2021-22  83.0     21       29   50  18.0  44.0
   number          last   first                 name  gp  goals  assists  pts 

In [221]:

curGoals=soup.find('section',{'id':'individual-overall-goaltenders'})
rows=curGoals.find_all('tr')
currGoalies=[]
for i in rows:
    col=i.find_all('td')
    name=i.find('span')
    
    if(name!=None):
        lastName,firstName=name.get_text().split(',')
        goalDict={'number':int(col[0].get_text()),
                   'last':lastName,
                   'first':firstName,
                   'name':firstName+' '+lastName,
                    'gp':int(col[1].get_text().split('-')[0]),
                    'mins':col[2].get_text(),
                    'ga':col[3].get_text(),
                    'gaa':col[4].get_text(),
                    'saves':col[8].get_text(),
                    'sv%':col[9].get_text(),
                    'W':col[10].get_text(),
                    'L':col[11].get_text(),
                    'T':col[12].get_text(),
                    'SO':col[13].get_text()}
        currGoalies.append(goalDict)


In [222]:
curGoaldf=pd.DataFrame(currGoalies)

In [226]:
curGoaldf[['number','name','gp','mins','ga','saves','sv%','gaa','W','L','T','SO']].to_string(index=False).split('\n')

[' number              name  gp     mins  ga saves   sv%   gaa   W   L  T SO',
 '     31   Vinny Duplessis  17   517:24  18   194  .915  2.09   6   2  0  2',
 '     29     Drew Commesso  28  1595:15  67   715  .914  2.52  13  11  3  1',
 '     30       Ashton Abel   5    00:00   0     0  .000   .00   0   0  0  0']

In [35]:
fileName=('SeasonLeaders.txt')
leadList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        leadSearch=re.search('(\d{4}-\d{2}) (\d*) (.*) (\d*) (.*) (\d*) (.*)',i)
        if leadSearch!=None:
            leadDict={'season':leadSearch.group(1),
                      'year': int(leadSearch.group(1)[:4])+1,
                       'goals':convertToInt(leadSearch.group(2)),
                        'gname':leadSearch.group(3),
                       'assists':convertToInt(leadSearch.group(4)),
                       'aname':leadSearch.group(5),
                       'pts':convertToInt(leadSearch.group(6)),
                       'pname':leadSearch.group(7)}
            leadList.append(leadDict)
f.close()
dfLead=pd.DataFrame(leadList)

In [27]:
fileName=('BeanpotHistory.txt')
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    beanList=[]
    for i in rows:
        col=i.split('\t')
        if(col[3]!=''):
            beanDict={"edition" : col[0],
                    "year" : col[1],
                    "arena" : col[2],
                    "semiDate" : col[3],
                    "semiAttend" : int(col[4]),
                    "semi1Winner" : col[5],
                    "semi1WinnerScore" : int(col[6]),
                    "semi1Loser" : col[7],
                    "semi1LoserScore" : int(col[8]),
                    "semi1OT" : col[9],
                    "semi2Winner" : col[10],
                    "semi2WinnerScore" : col[11],
                    "semi2Loser" : col[12],
                    "semi2LoserScore" : int(col[13]),
                    "semi2OT" : col[14],
                    "champDate" : col[15],
                    "champAttend" : int(col[16]),
                    "consWinner" : col[17],
                    "consWinnerScore" : int(col[18]),
                    "consLoser" : col[19],
                    "consLoserScore" : int(col[20]),
                    "consOT" : col[21],
                    "champion" : col[22],
                    "championScore" : int(col[23]),
                    "runnerup" : col[24],
                    "runnerupScore" : int(col[25]),
                    "champOT" : col[26]}
            beanDict['semiDOW'],beanDict['semiDate']=beanDict['semiDate'].split(',')
            beanDict['champDOW'],beanDict['champDate']=beanDict['champDate'].split(',')
            beanDict['semiDate']=beanDict['semiDate'].rstrip(' ').lstrip(' ')
            beanDict['champDate']=beanDict['champDate'].rstrip(' ').lstrip(' ')
            beanDict['semiDate']+='/'+beanDict['year']
            beanDict['champDate']+='/'+beanDict['year']
            beanDict['semiDate']=pd.Timestamp(beanDict['semiDate'])
            beanDict['champDate']=pd.Timestamp(beanDict['champDate'])
            beanList.append(beanDict)
f.close()
dfBeanpot=pd.DataFrame(beanList)

In [28]:
dfBeanpot.loc[(dfBeanpot['semiDate']>'2/1/1994')].groupby('champion').count()

,edition,year,arena,semiDate,semiAttend,semi1Winner,semi1WinnerScore,semi1Loser,semi1LoserScore,semi1OT,...,consWinnerScore,consLoser,consLoserScore,consOT,championScore,runnerup,runnerupScore,champOT,semiDOW,champDOW
champion,,,,,,,,,,,,,,,,,,,,,
Boston College,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
Boston University,14,14,14,14,14,14,14,14,14,14,...,14,14,14,14,14,14,14,14,14,14
Harvard,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Northeastern,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [38]:
url='http://www.collegehockeystats.net/0203/teamstats/bu_m'
f=urllib.request.urlopen(url)
html = f.read()
f.close()
soup = BeautifulSoup(html, 'html.parser')
table= soup.find("pre")


In [44]:
rows=table.get_text().split('\n')

In [191]:
seasons=["2002-03","2003-04","2004-05","2005-06","2006-07","2007-08","2008-09","2009-10","2010-11","2011-12","2012-13","2013-14"]
#seasons=["2014-15","2015-16","2016-17","2017-18","2018-19","2019-20","2020-21"]
playerList=[]
goalieList=[]
for season in seasons:
    sSeason=season[2:4]+season[-2:]
    url = "http://www.collegehockeystats.net/{}/teamstats/bu_m".format(sSeason)
    #print(url)
    f=urllib.request.urlopen(url)
    html = f.read()
    f.close()
    soup = BeautifulSoup(html, 'html.parser')
    table= soup.find("pre")
    rows=table.get_text().split('\n')
    goalies=False
    for i in rows:
        if('Goaltending (Overall)' in i):
            goalies=True
        if('Goaltending (Conference Only)' in i):
            break
        if('--' not in i and 'Boston University' not in i and 'Opponents' not in i and '##' not in i and 'Bench' not in i and not goalies):
            name,overall,conf,career=i.split('|')
            name=re.sub('\(.*\)','',name)
            if('Van der Gulik' in name):
                name=name.replace('Van der Gulik','VanderGulik')
            name=name.split(' ')
            overall=overall.split(' ')
            name=list(filter(None,name))
            overall=list(filter(None,overall))
            if('VanderGulik' in name[2]):
                name[2]=name[2].replace('VanderGulik','Van der Gulik')
            if(overall[4]=='0'):
                overall[4]+='/'
            playDict={'number':name[0],
                      'name':name[1]+ " " +name[2],
                      'pos':name[3],
                      'yr':name[4],
                      'gp':overall[0],
                      'goals':overall[1],
                      'assists':overall[2],
                      'pts':overall[3],
                      'pen':overall[4]+overall[5],
                      'season':season}
            playerList.append(playDict)
        if('----' not in i and 'Boston University' not in i and 'Opponents' not in i and '##' not in i and 'Open' not in i and goalies):
            name,overall,none=i.split('|')
            name=re.sub('\(.*\)','',name)
            name=name.split(' ')
            overall=overall.split(' ')
            name=list(filter(None,name))
            overall=list(filter(None,overall))
            if(len(overall)==13):
                goalieDict={'number':name[0],
                          'name':name[1]+ " " +name[2],
                          'yr':name[3],
                          'gp':overall[0],
                          'mins':overall[1],
                          'ga':overall[2],
                          'saves':overall[3],
                          'sv%':overall[5],
                          'gaa':overall[6],
                          'record':overall[7]+overall[8],
                          'SO':overall[11],
                          'season':season}
            else:
                goalieDict={'number':name[0],
                      'name':name[1]+ " " +name[2],
                      'yr':name[3],
                      'gp':overall[0],
                      'mins':overall[1],
                      'ga':overall[2],
                      'saves':overall[3],
                      'sv%':overall[5],
                      'gaa':overall[6],
                      'record':overall[7]+overall[8]+overall[9],
                      'SO':overall[11],
                      'season':season}
            goalieList.append(goalieDict)
dfPlay=pd.DataFrame(playerList)
dfGoal=pd.DataFrame(goalieList)

In [192]:
dfGoal

,number,name,yr,gp,mins,ga,saves,sv%,gaa,record,SO,season
0,31,Sean Fields,JR,40,2435:16,101,1035,.911,2.49,24-13-3,4,2002-03
1,1,Stephen Siwiec,FR,3,144:00,9,47,.839,3.75,1-1-0,2,2002-03
2,39,Andy Warren,JR,1,3:44,0,1,1.000,0.00,0-0-0,0,2002-03
3,31,Sean Fields,SR,34,1988:11,93,852,.902,2.81,10-16-7,1,2003-04
4,1,Stephan Siwiec,SO,7,327:34,13,141,.916,2.38,2-1-2,4,2003-04
5,39,John Curry,FR,1,5:10,0,2,1.000,0.00,0-0-0,0,2003-04
6,36,John Curry,SO,33,1949:40,64,769,.923,1.97,18-11-3,3,2004-05
7,1,Stephan Siwiec,JR,9,484:35,25,199,.888,3.10,5-2-1,8,2004-05
8,33,Karson Gillespie,FR,2,49:17,7,20,.741,8.52,0-1-0,1,2004-05
9,36,John Curry,JR,37,2166:01,81,907,.918,2.24,24-8-4,36,2005-06


In [197]:
dfGoalSeason[['number','name','yr','gp','mins','ga','saveperc','gaa','record','so','season']].to_string(index=False).split('\n')

,number,name,yr,gp,mins,ga,saveperc,gaa,record,so,season
0,29,Matt O'Connor,JR,35,2087:43,76,.927,2.18,25-4-4,1,2014-15
1,30,Connor LaCouvee,FR,8,417:53,18,.913,2.58,3-4-1,0,2014-15
2,1,Anthony Moccia,SR,2,4:21,0,1.000,0.00,0-0-0,0,2014-15
3,31,Sean Maguire,SR,25,1371:32,55,.920,2.41,13-9-1,1,2015-16
4,30,Connor LaCouvee,SO,19,986:12,48,.897,2.92,8-4-4,0,2015-16
5,29,Jake Oettinger,FR,35,2131:17,75,.927,2.11,21-11-3,4,2016-17
6,30,Connor LaCouvee,JR,5,244:03,11,.921,2.70,3-1-0,0,2016-17
7,29,Jake Oettinger,SO,38,2325:15,95,.915,2.45,21-13-4,5,2017-18
8,31,Max Prawdzik,JR,3,125:10,5,.914,2.40,1-1-0,1,2017-18
9,30,Jake Oettinger,JR,36,2110:06,86,.926,2.45,16-16-4,4,2018-19


In [227]:
x=dfPlay.to_string(index=False).split('\n')
x=dfSkateSeason[['number','name','pos','yr','overall_gp','overall_goals','overall_assists','overall_pts','overall_pen','season']].to_string(index=False).split('\n')
x=dfCurr[['number','name','gp','goals','assists','pts','pen']].to_string(index=False).split('\n')
x=dfGoal.to_string(index=False).split('\n')
x=dfGoalSeason[['number','name','yr','gp','mins','ga','saves','saveperc','gaa','record','so','season']].to_string(index=False).split('\n')
x=curGoaldf[['number','name','gp','mins','ga','saves','sv%','gaa','W','L','T','SO']].to_string(index=False).split('\n')

In [228]:
dfPlay.loc[dfPlay['name'].str.contains('Van')]

,number,name,pos,yr,gp,goals,assists,pts,pen,season
9,19,David Van der Gulik,F,FR,40,10,10,20,25/58,2002-03
27,7,David Van der Gulik,F,SO,35,13,7,20,29/74,2003-04
50,7,David Van der Gulik,F,JR,41,18,13,31,24/48,2004-05
82,7,David Van der Gulik,F,SR,25,11,11,22,13/26,2005-06


In [230]:
season='2021-22'
for i in x:
    i=i.split('  ')
    i=list(filter(None,i))
    for d in i:
        print(d,end='\t')
    print(season,end='')
    print()

 number	name	gp	 mins	ga saves	 sv%	 gaa	 W	 L	T SO	2021-22
 31	 Vinny Duplessis	17	 517:24	18	 194	.915	2.09	 6	 2	0	2	2021-22
 29	 Drew Commesso	28	1595:15	67	 715	.914	2.52	13	11	3	1	2021-22
 30	 Ashton Abel	 5	00:00	 0	 0	.000	 .00	 0	 0	0	0	2021-22


In [156]:
dfJersey

NameError: name 'dfJersey' is not defined